# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_to_more_columns(c_df, column_name, sport_df):
    
    # transform cities with more teams into more columns
    # Exclude Nan rows
    cities_sport_nonull = c_df[~c_df[column_name].isna()][column_name]
    
    # Exclude rows which team names are compose by two words
    teams = cities_sport_nonull[~cities_sport_nonull.str.contains("^([A-Z][a-z]+ )")]
    
    # map old team name to a list with team names
    teams = {s:re.findall(r"([A-Z\d][a-z]+\s[A-Z\d][a-z]+|[A-Z\d][a-z]*)", s) for s in teams}
    
    teams = {k:([k] if k not in teams.keys() else teams[k]) for k in c_df[column_name]}
    c_df[column_name] = c_df[column_name].map(teams)
    c_df = c_df.explode(column_name)
    
    # Exclude Nan rows
    cities_sport_nonull = c_df[~c_df[column_name].isna()][column_name]
    
    # Match Team names
    sport_df_team = sport_df["team"]

    # map cities team column to the sport_df team column
    row_mapping = {sport_t:t for sport_t in cities_sport_nonull for t in sport_df_team if sport_t in t}
    c_df[column_name].replace(row_mapping, inplace=True)
    
    return c_df
    
def manage_cities(c_df, column_name):
    
    # clean teams names from thing like "[note 30]"
    c_df[column_name] = c_df[column_name].str.replace(r"\[[\w ]*\]", "")
    
    # set '' as nan
    c_df.replace('', np.nan, inplace=True)
    
    # set '—' as nan
    c_df[column_name].replace('—', np.nan, inplace=True)
    
    # clean Population column name
    c_df.rename({"Population (2016 est.)[8]": "Population"}, axis='columns', inplace=True)
    return c_df

def manage_nhl_df(sport_df):
    # select only data from 2018
    sport_df = sport_df[sport_df["year"] == 2018]
    
    # keep only row which contains numbers in column "W"
    sport_df = sport_df[sport_df["W"].str.contains("\d+")]
    
    # remove asterisks from team names
    sport_df["team"] = sport_df["team"].str.replace('*', '')
    
    # Calcualate Win/Loss ratio
    sport_df["W/L"] = sport_df["W"].astype(int) / (sport_df["W"].astype(int) + sport_df["L"].astype(int))
    
    # select interested columns
    sport_df = sport_df[["team", "W/L"]]

    return sport_df
    

def nhl_correlation():
    global nhl_df
    global cities
    c_df = cities.copy()
    s_df = nhl_df.copy()

    s_df = manage_nhl_df(s_df)
    c_df = manage_cities(c_df, "NHL")
    c_df = cities_to_more_columns(c_df, "NHL", s_df)
    
    # rename NHL column
    c_df.rename({"NHL": "team"}, axis='columns', inplace=True)

    # merge the two datasets
    df = pd.merge(c_df, s_df, how='inner', on='team')
    
    # convert population column into int
    df["Population"] = df["Population"].astype(int)
    
    # calculate mean of W/L if more teams in a city
    df = df.groupby("Metropolitan area").mean()
    
    population_by_region = list(df["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region =  list(df["W/L"])# pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def manage_nba_df(sport_df):
    # select only data from 2018
    sport_df = sport_df[sport_df["year"] == 2018]
    
    # keep only row which contains numbers in column "W"
    sport_df = sport_df[sport_df["W"].str.contains("\d+")]
    
    # remove asterisks from team names
    sport_df["team"] = sport_df["team"].str.replace('*', '', regex=False)
    
    # clean team names from thing like "(1)"
    sport_df["team"] = sport_df["team"].str.replace(r'[\s]?\([\d]+\)', '', regex=True)
    
    # Calcualate Win/Loss ratio
    sport_df["W/L"] = sport_df["W"].astype(int) / (sport_df["L"].astype(int) + sport_df["W"].astype(int))
    
    # select interested columns
    sport_df = sport_df[["team", "W/L"]]
    return sport_df

def nba_correlation():
    global nba_df
    global cities
    c_df = cities.copy()
    s_df = nba_df.copy()
    
    s_df = manage_nba_df(s_df)
    c_df = manage_cities(c_df, "NBA")
    c_df = cities_to_more_columns(c_df, "NBA", s_df)
    
    # rename NHL column
    c_df.rename({"NBA": "team"}, axis='columns', inplace=True)
    
    # merge the two datasets
    df = pd.merge(c_df, s_df, how='inner', on='team')
    
    # convert population column into int
    df["Population"] = df["Population"].astype(int)
    
    # convert W/L column into float
    df["Population"] = df["Population"].astype(float)
    
    
    # calculate mean of W/L if more teams in a city
    df = df.groupby("Metropolitan area").mean()
    
    population_by_region = list(df["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region =  list(df["W/L"])# pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def manage_mlb_df(sport_df):
    # select only data from 2018
    sport_df = sport_df[sport_df["year"] == 2018]
    
    # remove asterisks from team names
    sport_df["team"] = sport_df["team"].str.replace('*', '', regex=False)
    
    # clean team names from thing like "(1)"
    sport_df["team"] = sport_df["team"].str.replace(r'[\s]?\([\d]+\)', '', regex=True)
    
    # Calcualate Win/Loss ratio
    sport_df["W/L"] = sport_df["W"] / (sport_df["L"] + sport_df["W"])
    
    # select interested columns
    sport_df = sport_df[["team", "W/L"]]
    return sport_df

def mlb_correlation(): 
    global mlb_df
    global cities
    c_df = cities.copy()
    s_df = mlb_df.copy()
    
    s_df = manage_mlb_df(s_df)
    c_df = manage_cities(c_df, "MLB")
    c_df = cities_to_more_columns(c_df, "MLB", s_df)
    
    # rename NHL column
    c_df.rename({"MLB": "team"}, axis='columns', inplace=True)
    
    # merge the two datasets
    df = pd.merge(c_df, s_df, how='inner', on='team')
    
    # convert population column into int
    df["Population"] = df["Population"].astype(int)
    
    # convert W/L column into float
    df["Population"] = df["Population"].astype(float)
    
    # calculate mean of W/L if more teams in a city
    df = df.groupby("Metropolitan area").mean()
    
    population_by_region = list(df["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region =  list(df["W/L"])# pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def manage_nfl_df(sport_df):
    # select only data from 2018
    sport_df = sport_df[sport_df["year"] == 2018]
    
    # keep only row which contains numbers in column "W"
    sport_df = sport_df[sport_df["W"].str.contains("\d+")]
    
    # remove asterisks from team names
    sport_df["team"] = sport_df["team"].str.replace('*', '', regex=False)
    
    # remove plus from team names
    sport_df["team"] = sport_df["team"].str.replace('+', '', regex=False)
    
    # clean team names from thing like "(1)"
    sport_df["team"] = sport_df["team"].str.replace(r'[\s]?\([\d]+\)', '', regex=True)
    
    # Calcualate Win/Loss ratio
    sport_df["W/L"] = sport_df["W"].astype(int) / (sport_df["L"].astype(int) + sport_df["W"].astype(int))
    
    # select interested columns
    sport_df = sport_df[["team", "W/L"]]
    return sport_df

def nfl_correlation(): 
    global nfl_df
    global cities
    c_df = cities.copy()
    s_df = nfl_df.copy()
    
    s_df = manage_nfl_df(s_df)
    c_df = manage_cities(c_df, "NFL")
    c_df = cities_to_more_columns(c_df, "NFL", s_df)
    
    # rename NHL column
    c_df.rename({"NFL": "team"}, axis='columns', inplace=True)
    
    # merge the two datasets
    df = pd.merge(c_df, s_df, how='inner', on='team')
    
    # convert population column into int
    df["Population"] = df["Population"].astype(int)
    
    # convert W/L column into float
    df["Population"] = df["Population"].astype(float)
    
    # calculate mean of W/L if more teams in a city
    df = df.groupby("Metropolitan area").mean()
    
    population_by_region = list(df["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region =  list(df["W/L"]) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_to_more_columns(cities_df, column_name, sport_df):
    # transform cities with more teams into more columns
    # Exclude Nan rows
    cities_sport_nonull = cities_df[~cities_df[column_name].isna()][column_name]
    
    # Exclude rows which team names are compose by two words
    teams = cities_sport_nonull[~cities_sport_nonull.str.contains("^([A-Z][a-z]+ )")]
    
    # map old team name to a list with team names
    teams = {s:re.findall(r"([A-Z\d][a-z]+\s[A-Z\d][a-z]+|[A-Z\d][a-z]*)", s) for s in teams}
    
    teams = {(k):([k] if k not in teams.keys() else teams[k]) for k in cities_df[column_name]}
    cities_df[column_name] = cities_df[column_name].map(teams)
    cities_df = cities_df.explode(column_name)
    
    # Exclude Nan rows
    cities_sport_nonull = cities_df[~cities_df[column_name].isna()][column_name]
    
    # Match Team names
    sport_df_team = sport_df["team"]

    # map cities team column to the sport_df team column
    row_mapping = {sport_t:t for sport_t in cities_sport_nonull for t in sport_df_team if sport_t in t}
    cities_df[column_name].replace(row_mapping, inplace=True)
    
    cities_df = cities_df[["Metropolitan area", "Population", column_name]]
    
    # rename column_name
    cities_df.rename({column_name: "team"}, axis='columns', inplace=True)
    
    return cities_df
    
def manage_cities(cities_df, column_name):
    # clean teams names from thing like "[note 30]"
    cities_df[column_name] = cities_df[column_name].str.replace(r"\[[\w ]*\]", "")
    
    # set '' as nan
    cities_df.replace('', np.nan, inplace=True)
    
    # set '—' as nan
    cities_df[column_name].replace('—', np.nan, inplace=True)
    
    # clean Population column name
    cities_df.rename({"Population (2016 est.)[8]": "Population"}, axis='columns', inplace=True)
    
    return cities_df

def sports_team_performance():
    global cities
    global mlb_df
    global nhl_df
    global nba_df
    global nfl_df
    
    # copy cities df
    mlb_cities = cities.copy()
    nhl_cities = cities.copy()
    nba_cities = cities.copy()
    nfl_cities = cities.copy()
    
    # copy sports df
    df_mlb = mlb_df.copy()
    df_nhl = nhl_df.copy()
    df_nba = nba_df.copy()
    df_nfl = nfl_df.copy()
    
    
    # clean each sport df
    df_mlb = manage_mlb_df(df_mlb)
    df_nhl = manage_nhl_df(df_nhl)
    df_nba = manage_nba_df(df_nba)
    df_nfl = manage_nfl_df(df_nfl)
    
    # manage mlb_cities df
    mlb_cities = manage_cities(mlb_cities, "MLB")
    mlb_cities = cities_to_more_columns(mlb_cities, "MLB", df_mlb)
    
    # manage nhl_cities df
    nhl_cities = manage_cities(nhl_cities, "NHL")
    nhl_cities = cities_to_more_columns(nhl_cities, "NHL", df_nhl)
    
    # manage nba_cities df
    nba_cities = manage_cities(nba_cities, "NBA")
    nba_cities = cities_to_more_columns(nba_cities, "NBA", df_nba)
    
    # manage nfl_cities df
    nfl_cities = manage_cities(nfl_cities, "NFL")
    nfl_cities = cities_to_more_columns(nfl_cities, "NFL", df_nfl)
    
    # merge the mlb datasets
    final_mlb = pd.merge(mlb_cities, df_mlb, how='inner', on='team')
    # delete Population Column
    del final_mlb["Population"]
    
    # merge the nhl datasets
    final_nhl = pd.merge(nhl_cities, df_nhl, how='inner', on='team')
    # delete Population Column
    del final_nhl["Population"]
    
    # merge the nba datasets
    final_nba = pd.merge(nba_cities, df_nba, how='inner', on='team')
    # delete Population Column
    del final_nba["Population"]
    
    # merge the nfl datasets
    final_nfl = pd.merge(nfl_cities, df_nfl, how='inner', on='team')
    # delete Population Column
    del final_nfl["Population"]
    
    
    # merge datasets for different sports
    nfl_nba = pd.merge(final_nfl, final_nba, how='inner', on="Metropolitan area", suffixes=('_nfl', '_nba'))
    # calculate mean of W/L if more teams in a city
    nfl_nba = nfl_nba.groupby("Metropolitan area").mean()
    
    nfl_nhl = pd.merge(final_nfl, final_nhl, how='inner', on="Metropolitan area", suffixes=('_nfl', '_nhl'))
    # calculate mean of W/L if more teams in a city
    nfl_nhl = nfl_nhl.groupby("Metropolitan area").mean()
    
    nfl_mlb = pd.merge(final_nfl, final_mlb, how='inner', on="Metropolitan area", suffixes=('_nfl', '_mlb'))
    # calculate mean of W/L if more teams in a city
    nfl_mlb = nfl_mlb.groupby("Metropolitan area").mean()
    
    nba_nhl = pd.merge(final_nba, final_nhl, how='inner', on="Metropolitan area", suffixes=('_nba', '_nhl'))
    # calculate mean of W/L if more teams in a city
    nba_nhl = nba_nhl.groupby("Metropolitan area").mean()
    
    nba_mlb = pd.merge(final_nba, final_mlb, how='inner', on="Metropolitan area", suffixes=('_nba', '_mlb'))
    # calculate mean of W/L if more teams in a city
    nba_mlb = nba_mlb.groupby("Metropolitan area").mean()
    
    nhl_mlb = pd.merge(final_nhl, final_mlb, how='inner', on="Metropolitan area", suffixes=('_nhl', '_mlb'))
    # calculate mean of W/L if more teams in a city
    nhl_mlb = nhl_mlb.groupby("Metropolitan area").mean()
    
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    # Calculate p-values
    p_values.loc['NFL', 'NBA'] = p_values.loc['NBA', 'NFL'] = stats.ttest_rel(list(nfl_nba["W/L_nfl"]), list(nfl_nba["W/L_nba"])).pvalue
    p_values.loc['NFL', 'NHL'] = p_values.loc['NHL', 'NFL'] = stats.ttest_rel(list(nfl_nhl["W/L_nfl"]), list(nfl_nhl["W/L_nhl"])).pvalue
    p_values.loc['NFL', 'MLB'] = p_values.loc['MLB', 'NFL'] = stats.ttest_rel(list(nfl_mlb["W/L_nfl"]), list(nfl_mlb["W/L_mlb"])).pvalue
    p_values.loc['NBA', 'NHL'] = p_values.loc['NHL', 'NBA'] = stats.ttest_rel(list(nba_nhl["W/L_nba"]), list(nba_nhl["W/L_nhl"])).pvalue
    p_values.loc['NBA', 'MLB'] = p_values.loc['MLB', 'NBA'] = stats.ttest_rel(list(nba_mlb["W/L_nba"]), list(nba_mlb["W/L_mlb"])).pvalue
    p_values.loc['NHL', 'MLB'] = p_values.loc['MLB', 'NHL'] = stats.ttest_rel(list(nhl_mlb["W/L_nhl"]), list(nhl_mlb["W/L_mlb"])).pvalue
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values